In [1]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras as kr
from sklearn import preprocessing
import tensorflow as tf
import re

Using TensorFlow backend.


In [2]:
# open and read the id-list
df = pd.read_csv('code.csv')
print(df)

                                               text_code      sent  \
0      [    3     1     0     2  5717    31   108 108...  negative   
1      [    3     0   332     0    38     2   934   2...  negative   
2      [   1    0    2    7  315    0    0 1144    0 ...  negative   
3      [    1     0     0  1373  4294   253    41    ...  negative   
4      [    0     1    10    17   712     0  2404    ...  negative   
...                                                  ...       ...   
10546  [  11    0    0    0  534    0    0  263    0 ...  positive   
10547  [  12    1  550    0   22   46 6911  121   14 ...  positive   
10548  [   0   61    0   92  121   14    0  133 6174 ...  positive   
10549  [    0     9    10   335     0     0   183   2...  positive   
10550  [   0    1    0  638    0    0  177  247    9 ...  positive   

             topic  
0      amy schumer  
1      amy schumer  
2      amy schumer  
3      amy schumer  
4      amy schumer  
...            ...  
10546       

In [3]:
# type-change function
def textcode_to_array(textcode):
    listOfTokens = re.split(r'\W+',textcode)
    # decrease the length of array since there are spaces at the start and end of the splitted list
    codes = np.zeros(len(listOfTokens)-2, dtype=np.int) 
    for i in range(len(codes)):
        codes[i] = int(listOfTokens[i+1])
    return codes

In [4]:
# do the type change
df['code'] = ""
for i in range(df.shape[0]):
    df.iloc[i,3] =  textcode_to_array(df.iloc[i,0])
print(df)

                                               text_code      sent  \
0      [    3     1     0     2  5717    31   108 108...  negative   
1      [    3     0   332     0    38     2   934   2...  negative   
2      [   1    0    2    7  315    0    0 1144    0 ...  negative   
3      [    1     0     0  1373  4294   253    41    ...  negative   
4      [    0     1    10    17   712     0  2404    ...  negative   
...                                                  ...       ...   
10546  [  11    0    0    0  534    0    0  263    0 ...  positive   
10547  [  12    1  550    0   22   46 6911  121   14 ...  positive   
10548  [   0   61    0   92  121   14    0  133 6174 ...  positive   
10549  [    0     9    10   335     0     0   183   2...  positive   
10550  [   0    1    0  638    0    0  177  247    9 ...  positive   

             topic                                               code  
0      amy schumer  [3, 1, 0, 2, 5717, 31, 108, 10841, 1585, 10840...  
1      amy schu

In [5]:
le = preprocessing.LabelEncoder() 
df['topicnum'] = le.fit_transform(df.iloc[:,2])

In [6]:
df = df[['sent','code','topicnum']]

In [7]:
def LSTM(x,weights,bias):
    embedding_inputs = tf.nn.embedding_lookup(embedding, x) # replace the integers in id list into embedding vectors
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(num_units) # generate a LSTM model with num_units cells in each layer
    
    outputs, states = tf.nn.dynamic_rnn(lstm_cell,embedding_inputs, dtype=tf.float32) # run the model and get the last output
    outputs = tf.nn.dropout(outputs, keep_prob)     # dropout process
    result = tf.nn.softmax(tf.matmul(outputs[:,-1,:],weights)+bias,1)  # calculate the probability vector

    return result

In [8]:
trainaccuracylist = np.zeros(100)
valaccuracylist = np.zeros(100)
testaccuracylist = np.zeros(100)
num_classes = 2 # positive, negative
for i in range(100):
    df2 = df.loc[df['topicnum']==i]
    # use labelencoder to change the type of classes from String to integer
    labels = le.fit_transform(df2.iloc[:,0])
    # and then, change the type from integer to one-hot code
    labels = kr.utils.to_categorical(labels, num_classes) 
    X_trainval,X_test,y_trainval,y_test=train_test_split(df2.iloc[:,1],labels,test_size=0.2,random_state=1) 
    X_train,X_val,y_train,y_val=train_test_split(X_trainval,y_trainval,test_size=0.25,random_state=1) 
    num_units = 16
    batch_size=30
    n_batch = int(len(X_train.shape)/batch_size)
    train_rate = 0.0001
    embedding_size = 16
    vocabulary_size = 13465
    x=tf.placeholder(tf.int32,shape=[None,66]) #
    y=tf.placeholder(tf.float32,[None,num_classes])
    weights=tf.Variable(tf.truncated_normal([num_units,num_classes],stddev=0.1))
    bias=tf.Variable(tf.constant(0.1,shape=[num_classes]))
    embedding = tf.get_variable('embedding', [vocabulary_size, embedding_size])
    keep_prob = tf.placeholder(tf.float32)
    prediction=LSTM(x,weights,bias) # get the probability vector
    cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))  # calculate the cost
    train_step=tf.train.AdamOptimizer(train_rate).minimize(cost) # use optimizer to improve the model in a given rate
    correct_predict=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) # generate accuracy vector
    accuracy=tf.reduce_mean(tf.cast(correct_predict,tf.float32)) # calculate accuracy

    init=tf.global_variables_initializer() # initialize all variables

    # run the training function
    with tf.Session() as sess:
        sess.run(init)
        for j in range(n_batch):
            TrainData_label=y_train[j*batch_size:(j+1)*n_batch]
            TrainData_batch = kr.preprocessing.sequence.pad_sequences(X_train.iloc[j*batch_size:(j+1)*n_batch], 66) # increase the sequence length to maxlen
            sess.run(train_step,feed_dict={x:TrainData_batch,y:TrainData_label, keep_prob: 0.75})
    TrainData = kr.preprocessing.sequence.pad_sequences(X_train, 66)
    trainaccuracylist[i]=sess.run(accuracy, feed_dict={x: TrainData, y: y_train, keep_prob: 1}) # Prediction does not need dropout, so 1 is set
    ValData = kr.preprocessing.sequence.pad_sequences(X_val, 66)
    valaccuracylist[i] = sess.run(accuracy, feed_dict={x: ValData, y: y_val, keep_prob: 1})
    TestData = kr.preprocessing.sequence.pad_sequences(X_test, 66)
    testaccuracylist[i] = sess.run(accuracy, feed_dict={x: TestData, y: y_test, keep_prob: 1})

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



RuntimeError: Attempted to use a closed Session.